Code to detect small circles and measure their area and radius

In [1]:
import cv2
import numpy as np
import os
import zipfile
import pandas as pd
from google.colab import files

# Conversion factor from pixels to micrometers
PIXELS_TO_MICROMETERS = 9.3  # 9.3 pixels = 1 micrometer

# Specify the path to the single image
image_path = '/content/random_circles_inverted.png'  # Replace with the path to your image

# Create output directories
image_dir = os.path.dirname(image_path)
thresholded_dir = os.path.join(image_dir, 'thresholded_images')
contour_dir = os.path.join(image_dir, 'contour_images')
excel_dir = os.path.join(image_dir, 'excel_files')
os.makedirs(thresholded_dir, exist_ok=True)
os.makedirs(contour_dir, exist_ok=True)
os.makedirs(excel_dir, exist_ok=True)

# Function to process and analyze a single image
def process_and_analyze_image(image_path, thresholded_path, contour_path, excel_path):
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply adaptive thresholding
    adaptive_thresh = cv2.adaptiveThreshold(image, 255,
                                            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY_INV,
                                            11, 2)

    # Save thresholded image with higher compression (JPEG/PNG)
    if thresholded_path.endswith('.jpg') or thresholded_path.endswith('.jpeg'):
        cv2.imwrite(thresholded_path, adaptive_thresh, [int(cv2.IMWRITE_JPEG_QUALITY), 50])  # 50% quality for higher compression
    elif thresholded_path.endswith('.png'):
        cv2.imwrite(thresholded_path, adaptive_thresh, [int(cv2.IMWRITE_PNG_COMPRESSION), 9])  # Highest compression for PNG

    # Define kernel for morphological operations
    kernel = np.ones((5, 5), np.uint8)
    cleaned_image = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel)

    # Find contours
    contours, _ = cv2.findContours(cleaned_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Analyze contours
    data = []
    for contour in contours:
        area_pixels = cv2.contourArea(contour)
        if area_pixels > 10:  # Filter out small noise
            area_micrometers = area_pixels / (PIXELS_TO_MICROMETERS**2)  # Convert area to µm²
            (x, y), radius_pixels = cv2.minEnclosingCircle(contour)
            radius_micrometers = radius_pixels / PIXELS_TO_MICROMETERS  # Convert radius to µm
            data.append({
                "Area (µm²)": area_micrometers,
                "Center X (µm)": x / PIXELS_TO_MICROMETERS,
                "Center Y (µm)": y / PIXELS_TO_MICROMETERS,
                "Radius (µm)": radius_micrometers
            })

    # Save data to an Excel file
    df = pd.DataFrame(data)
    df.to_excel(excel_path, index=False)

    # Draw contours on the image
    result_image = cv2.cvtColor(cleaned_image, cv2.COLOR_GRAY2BGR)
    for row in data:
        center_x = int(row["Center X (µm)"] * PIXELS_TO_MICROMETERS)
        center_y = int(row["Center Y (µm)"] * PIXELS_TO_MICROMETERS)
        radius = int(row["Radius (µm)"] * PIXELS_TO_MICROMETERS)
        cv2.circle(result_image, (center_x, center_y), radius, (0, 255, 0), 2)

    # Save image with contours with higher compression (JPEG/PNG)
    if contour_path.endswith('.jpg') or contour_path.endswith('.jpeg'):
        cv2.imwrite(contour_path, result_image, [int(cv2.IMWRITE_JPEG_QUALITY), 50])  # 50% quality for higher compression
    elif contour_path.endswith('.png'):
        cv2.imwrite(contour_path, result_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 9])  # Highest compression for PNG

# Set the paths for output files
filename = os.path.basename(image_path)
thresholded_path = os.path.join(thresholded_dir, f'thresholded_{filename}')
contour_path = os.path.join(contour_dir, f'contour_{filename}')
excel_path = os.path.join(excel_dir, f'{os.path.splitext(filename)[0]}_analysis.xlsx')

# Process the single image
process_and_analyze_image(image_path, thresholded_path, contour_path, excel_path)

# Zip all output files for download
output_zip = 'processed_single_image_results.zip'
with zipfile.ZipFile(output_zip, 'w') as zipf:
    for file_path in [thresholded_path, contour_path, excel_path]:
        zipf.write(file_path, os.path.relpath(file_path, image_dir))

# Provide download link for the zip file
files.download(output_zip)

# Feedback on completion
print(f"Processed results for {filename} are saved and zipped as {output_zip}.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processed results for random_circles_inverted.png are saved and zipped as processed_single_image_results.zip.
